# Navier-Stokes No Estacionario con MMfem

Este notebook demuestra cómo resolver problemas evolutivos de Navier-Stokes usando MMfem.

## Configuración

In [ ]:
from ngsolve import CF, Integrate, InnerProduct, sqrt, Draw
from mmfem import (
    rectangle_mesh, taylor_hood,
    stokes_problem,
    time_stepping_semiimplicit,
    time_stepping_implicit
)
import matplotlib.pyplot as plt
import numpy as np

# Configuración de matplotlib
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## 1. Configuración del Problema

Vamos a resolver el clásico problema de cavity con tapa móvil.

In [ ]:
# Parámetros del problema
viscosity = 0.01  # ν (Re = 100)
h = 0.05          # Tamaño de malla
dt = 0.01         # Paso de tiempo
T_final = 2.0     # Tiempo final

print(f"Reynolds number: {1.0/viscosity}")
print(f"Time steps: {int(T_final/dt)}")

## 2. Crear Malla y Espacio de Elementos Finitos

In [ ]:
# Malla del dominio [0,1] × [0,1]
mesh = rectangle_mesh(0, 1, 0, 1, h=h)

# Espacio de Taylor-Hood (P2-P1)
X = taylor_hood(mesh, "left|right|bottom|top")

print(f"Elementos: {mesh.ne}")
print(f"DOFs totales: {X.ndof}")
print(f"DOFs velocidad: {X.components[0].ndof}")
print(f"DOFs presión: {X.components[1].ndof}")

## 3. Condiciones de Frontera e Inicial

In [ ]:
# Condición de frontera: tapa móvil en y=1
u_bc = CF((1, 0))

# Condición inicial: solución de Stokes
print("Computando condición inicial (Stokes)...")
stokes_sol = stokes_problem(mesh, X, "top", u_bc, viscosity)
u0 = stokes_sol.components[0]
print("✓ Condición inicial lista")

## 4. Evolución Temporal: Método Semi-Implícito

Primero probamos el método semi-implícito (rápido, requiere dt pequeño).

In [ ]:
# Lista para guardar energía cinética
ke_history = []

def save_kinetic_energy(time, timestep, solution):
    """Callback para guardar energía cinética."""
    velocity = solution.components[0]
    ke = 0.5 * Integrate(InnerProduct(velocity, velocity), mesh)
    ke_history.append((time, ke))

# Ejecutar simulación
print("\nIniciando simulación semi-implícita...")
solutions, times = time_stepping_semiimplicit(
    mesh=mesh,
    fespace=X,
    initial_velocity=u0,
    dirichlet_boundaries="top",
    velocity_bc=u_bc,
    dt=dt,
    T_final=T_final,
    viscosity=viscosity,
    save_frequency=10,
    verbose=True,
    callback=save_kinetic_energy
)

print(f"\n✓ Simulación completa: {len(solutions)} snapshots guardados")

## 5. Visualización de Resultados

In [ ]:
# Gráfico de energía cinética vs tiempo
if ke_history:
    ke_times, ke_values = zip(*ke_history)
    
    plt.figure()
    plt.plot(ke_times, ke_values, 'b-', linewidth=2, label='Energía Cinética')
    plt.xlabel('Tiempo')
    plt.ylabel('Energía Cinética')
    plt.title('Evolución de la Energía Cinética')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f"Energía cinética inicial: {ke_values[0]:.6f}")
    print(f"Energía cinética final: {ke_values[-1]:.6f}")
    print(f"Cambio: {(ke_values[-1]-ke_values[0])/ke_values[0]*100:.2f}%")

## 6. Visualización de la Solución en Diferentes Tiempos

In [ ]:
# Seleccionar 4 snapshots para visualizar
n_snapshots = min(4, len(solutions))
snapshot_indices = np.linspace(0, len(solutions)-1, n_snapshots, dtype=int)

print("Visualizando soluciones en tiempos:")
for idx in snapshot_indices:
    t = times[idx]
    sol = solutions[idx]
    velocity = sol.components[0]
    pressure = sol.components[1]
    
    print(f"  t = {t:.3f}")
    
    # Nota: Draw() abre una ventana GUI de NGSolve
    # Para visualización en notebook, considerar usar VTKOutput
    # o exportar a formatos soportados

## 7. Comparación: Método Totalmente Implícito

Ahora probamos el método totalmente implícito con un paso de tiempo más grande.

In [ ]:
# Reiniciar historia de energía
ke_history_implicit = []

def save_ke_implicit(time, timestep, solution):
    velocity = solution.components[0]
    ke = 0.5 * Integrate(InnerProduct(velocity, velocity), mesh)
    ke_history_implicit.append((time, ke))

# Usar paso de tiempo más grande
dt_implicit = 0.05  # 5x más grande!

print(f"\nIniciando simulación totalmente implícita con dt={dt_implicit}...")
solutions_impl, times_impl = time_stepping_implicit(
    mesh=mesh,
    fespace=X,
    initial_velocity=u0,
    dirichlet_boundaries="top",
    velocity_bc=u_bc,
    dt=dt_implicit,
    T_final=T_final,
    viscosity=viscosity,
    method='picard',
    tolerance=1e-8,
    max_nonlinear_iter=20,
    save_frequency=2,
    verbose=True,
    callback=save_ke_implicit
)

print(f"\n✓ Simulación implícita completa")

## 8. Comparación de Ambos Métodos

In [ ]:
# Comparar evolución de energía cinética
if ke_history and ke_history_implicit:
    ke_t_semi, ke_v_semi = zip(*ke_history)
    ke_t_impl, ke_v_impl = zip(*ke_history_implicit)
    
    plt.figure()
    plt.plot(ke_t_semi, ke_v_semi, 'b-', linewidth=2, 
             marker='o', markersize=4, label=f'Semi-implícito (dt={dt})')
    plt.plot(ke_t_impl, ke_v_impl, 'r--', linewidth=2, 
             marker='s', markersize=4, label=f'Totalmente implícito (dt={dt_implicit})')
    plt.xlabel('Tiempo')
    plt.ylabel('Energía Cinética')
    plt.title('Comparación de Métodos')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print("\nComparación de métodos:")
    print(f"Semi-implícito: {len(solutions)} snapshots, dt={dt}")
    print(f"Totalmente implícito: {len(solutions_impl)} snapshots, dt={dt_implicit}")
    print(f"\nAmbos métodos deberían converger a la misma solución.")

## 9. Análisis del Campo de Velocidad

In [ ]:
# Analizar la solución final
final_solution = solutions[-1]
final_velocity = final_solution.components[0]
final_pressure = final_solution.components[1]

# Calcular magnitud de velocidad promedio
vel_magnitude = sqrt(InnerProduct(final_velocity, final_velocity))
avg_vel = Integrate(vel_magnitude, mesh) / Integrate(1, mesh)

print(f"\nEstadísticas de la solución final (t={times[-1]:.3f}):")
print(f"  Velocidad promedio: {avg_vel:.6f}")
print(f"  Energía cinética: {ke_values[-1]:.6f}")

## 10. Exportación para Visualización Externa

In [ ]:
# Exportar a VTK para visualización en ParaView
from ngsolve import VTKOutput

print("Exportando soluciones a formato VTK...")
vtk = VTKOutput(
    ma=mesh,
    coefs=[sol.components[0] for sol in solutions],
    names=["velocity"],
    filename="unsteady_cavity_notebook",
    subdivision=2
)
vtk.Do()

print("✓ Archivos VTK generados")
print("  Visualizar con: paraview unsteady_cavity_notebook.vtu")

## Conclusiones

En este notebook hemos:

1. ✓ Configurado un problema evolutivo de Navier-Stokes
2. ✓ Resuelto usando método semi-implícito (rápido, dt pequeño)
3. ✓ Resuelto usando método totalmente implícito (estable, dt grande)
4. ✓ Comparado ambos métodos
5. ✓ Analizado la evolución temporal de la solución
6. ✓ Exportado resultados para visualización

### Próximos Pasos

- Experimentar con diferentes números de Reynolds
- Probar diferentes geometrías
- Implementar refinamiento adaptativo
- Estudiar convergencia temporal